# Spatially Enabled DataFrame の概要

[`Spatially Enabled DataFrame`](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.features.toc.html#spatialdataframe) (SEDF) は、属性データなどを簡単に操作できるシンプルで直感的なオブジェクトを作成します。
    
<blockquote>
    バージョン 1.5 で追加された Spatially Enabled DataFrame は、これまでの SpatialDataFrame オブジェクトを進化させたものです。SDF オブジェクトはまだ使用可能ですが、チームは積極的な開発を中止し、この新しい Spatially Enabled DataFrame パターンの使用を促進しています。SEDF は、より優れたメモリ管理、より大きなデータセットを扱う能力を提供し、Pandas が今後の道筋として提唱しているパターンです。</blockquote>

Spatially Enabled DataFrame は、一般的な [Pandas](https://pandas.pydata.org/) [DataFrame](http://pandas.pydata.org/pandas-docs/stable/dsintro.html#dataframe) の構造に spatial というカスタムネームスペースを挿入して、空間的な機能を持たせます。これにより、属性列と空間列の両方に対して、直感的な操作を使用することができます。

データフレームは、シェープファイル、Pandas DataFrames、フィーチャ クラス、GeoJSON、フィーチャ レイヤーなど、多くのソースから読み込むことができます。


このドキュメントでは、GIS データを操作するために Spatially Enabled DataFrame オブジェクトを使用するための基本的な方法について説明します。

* [GIS データへのアクセス](#GIS-データへのアクセス)
 * [Web レイヤーの読み取り](#Web-レイヤーの読み取り)
 * [フィーチャ レイヤーの読み取り](#フィーチャ-レイヤーの読み取り)
   * [例：フィーチャ レイヤーからデータフレームを取得](#例：フィーチャ-レイヤーからデータフレームを取得)
   * [例：フィーチャ レイヤーのクエリ結果を Spatially Enabled DataFrame に送信](#例：フィーチャ-レイヤーのクエリ結果をSpatially-Enabled-DataFrame-に送信する)
* [ローカルの GIS データへのアクセス](#ローカルの-GIS-データへのアクセス)
 * [例：シェープファイルの読み込み](#例：シェープファイルの読み込み)
 * [例：ファイルジオデータベースからフィーチャ クラスを読み込む](#例：ファイルジオデータベースからフィーチャ-クラスを読み込む)
* [Spatially Enabled DataFrame の保存](#Spatially-Enabled-DataFrame-の保存)
 * [エクスポート オプション](#エクスポート-オプション)
 * [フィーチャ クラスへのエクスポート](#フィーチャ-クラスへのエクスポート)
   * [例：データセット全体をシェープファイルにエクスポート](#例：データセット全体をシェープファイルにエクスポート)
   * [例：データセットをファイルジオデータベースのフィーチャ クラスにエクスポート](#例：データセットをファイルジオデータベースのフィーチャ-クラスにエクスポート)
   * [例：列のサブセットと上位5つのレコードを含むデータセットをシェープファイルにエクスポート](#例：列のサブセットと上位5つのレコードを含むデータセットをシェープファイルにエクスポート)
 * [フィーチャ レイヤーとして公開](#フィーチャ-レイヤーとして公開)
    * [例：フィーチャ レイヤーとして公開](#例：フィーチャ-レイヤーとして公開)
   
   

In [1]:
import pandas as pd
from arcgis.features import GeoAccessor, GeoSeriesAccessor

## GIS データへのアクセス

GIS ユーザーは、ポータル上に公開されているレイヤー（Web レイヤー）とローカル データの両方で作業する必要がありますが、データを永続的にコピーすることなく、これらのデータセットを操作する機能が不足しています。Spatial Enabled DataFrame は、地理空間データの読み取り、書き込み、操作が可能なインメモリ・オブジェクトであるため、この問題を解決することができます。

SEDF は、Esri の [ArcPy サイト パッケージ](http://pro.arcgis.com/en/pro-app/arcpy/get-started/what-is-arcpy-.htm)、オープン ソースの [pyshp](https://github.com/GeospatialPython/pyshp/)、[shapely](https://github.com/Toblerity/Shapely)、および [fiona](https://github.com/Toblerity/Fiona) パッケージと統合されています。つまり、ArcGIS API for Python の SEDF は、これらのジオメトリ エンジンのいずれかを使用して、プラットフォームに関係なく地理空間データを簡単に操作するためのオプションを提供することができます。SEDF はデータを希望の形式に変換するので、Python の機能を使用して地理情報の分析や可視化を行うことができます。

データを読み込んでスクリプト化してワークフローを自動化したり、JupyterNotebook のマップ上に簡単に可視化したりすることができます。また、データをフィーチャ クラスとしてエクスポートしたり、必要に応じてサーバーに直接公開して共有することができます。汎用性の高い Spatial Enabled DataFrame で利用できるさまざまなオプションのいくつかを探ってみましょう。

### Web レイヤーの読み取り

[ArcGIS Online](https://www.arcgis.com) や [ArcGIS Enterprise](http://enterprise.arcgis.com) でホストされている[フィーチャ レイヤー](https://doc.arcgis.com/en/arcgis-online/share-maps/hosted-web-layers.htm)は、[from_layer](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.features.toc.html?highlight=from_layer#arcgis.features.GeoAccessor.from_layer) メソッドを使用して Spatially Enabled DataFrame に簡単に読み込むことができます。SEDF オブジェクトに読み込んだら、レポートを作成したり、データを操作したり、目的に合った形式に変換したりすることができます。

例：ArcGIS Online の[アイテム](https://developers.arcgis.com/rest/users-groups-and-items/publish-item.htm)を取得し、[layers](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.gis.toc.html#layer) プロパティを使用してレイヤー
の最初の 5 つのレコードを表示します。

In [2]:
from arcgis import GIS
gis = GIS()
item = gis.content.get("85d0ca4ea1ca4b9abf0c51b9bd34de2e")
flayer = item.layers[0]

# Spatially Enabled DataFrame オブジェクトの作成
sdf = pd.DataFrame.spatial.from_layer(flayer)
sdf.head()

,AGE_10_14,AGE_15_19,AGE_20_24,AGE_25_34,AGE_35_44,AGE_45_54,AGE_55_64,AGE_5_9,AGE_65_74,AGE_75_84,...,PLACEFIPS,POP2010,POPULATION,POP_CLASS,RENTER_OCC,SHAPE,ST,STFIPS,VACANT,WHITE
0,1313,1058,734,2031,1767,1446,1136,1503,665,486,...,1601990,13816,15181,6,1271,"{""x"": -12462673.723706165, ""y"": 5384674.994080...",ID,16,271,13002
1,890,817,818,1799,1235,1330,1143,1099,721,579,...,1607840,11899,11946,6,1441,"{""x"": -12506251.313993266, ""y"": 5341537.793529...",ID,16,318,9893
2,12750,13959,16966,32135,27048,29595,24177,12933,12176,7087,...,1608830,205671,225405,8,33359,"{""x"": -12938676.6836459, ""y"": 5403597.04949123...",ID,16,6996,182991
3,790,768,699,1445,1136,1134,935,959,679,464,...,1611260,10345,10727,6,1461,"{""x"": -12667411.402393516, ""y"": 5241722.820606...",ID,16,241,7984
4,3803,3779,3687,7571,5559,4744,3624,4397,2296,1222,...,1612250,46237,53942,7,5196,"{""x"": -12989383.674504515, ""y"": 5413226.487333...",ID,16,1428,35856


オブジェクトの型を調べると、標準のpandas DataFrameオブジェクトが返ってきます。しかし、このオブジェクトには、幾何学的な操作を実行できるSHAPE列が追加されています。

In [3]:
type(sdf)

pandas.core.frame.DataFrame

さらに、DataFrame には、オブジェクトに対して実行可能なジオプロセシング操作のリストを提供する新しい空間プロパティがあります。このセクションの残りのガイドでは、これらの機能の使用方法について詳しく説明します。

### フィーチャ レイヤーの読み取り

上で見たように、SEDF は、ArcGIS Online や ArcGIS Enterprise でホストされるフィーチャ レイヤーを使用することができます。ここでは、上記のノートブック セルを分解して、フィーチャ レイヤーからレコードのサブセットを抽出する方法を説明します。

#### 例：フィーチャ レイヤーからデータフレームを取得

SEDF の from_layer メソッドを使用して、アイテムのレイヤーからデータフレームをインスタンス化し、最初の5つのレコードを取得します。

In [4]:
# アイテムID でフィーチャ レイヤーを取得
known_item = gis.content.get("85d0ca4ea1ca4b9abf0c51b9bd34de2e")
known_item

<Item title:"USA Major Cities" type:Feature Layer Collection owner:esri_dm>

In [5]:
# アイテムの1番目のレイヤーを取得
fl = known_item.layers[0]

# from_layer メソッドを使用します
sdf = pd.DataFrame.spatial.from_layer(fl)

# 最初の5つのレコードを返します
sdf.head()

,AGE_10_14,AGE_15_19,AGE_20_24,AGE_25_34,AGE_35_44,AGE_45_54,AGE_55_64,AGE_5_9,AGE_65_74,AGE_75_84,...,PLACEFIPS,POP2010,POPULATION,POP_CLASS,RENTER_OCC,SHAPE,ST,STFIPS,VACANT,WHITE
0,1313,1058,734,2031,1767,1446,1136,1503,665,486,...,1601990,13816,15181,6,1271,"{""x"": -12462673.723706165, ""y"": 5384674.994080...",ID,16,271,13002
1,890,817,818,1799,1235,1330,1143,1099,721,579,...,1607840,11899,11946,6,1441,"{""x"": -12506251.313993266, ""y"": 5341537.793529...",ID,16,318,9893
2,12750,13959,16966,32135,27048,29595,24177,12933,12176,7087,...,1608830,205671,225405,8,33359,"{""x"": -12938676.6836459, ""y"": 5403597.04949123...",ID,16,6996,182991
3,790,768,699,1445,1136,1134,935,959,679,464,...,1611260,10345,10727,6,1461,"{""x"": -12667411.402393516, ""y"": 5241722.820606...",ID,16,241,7984
4,3803,3779,3687,7571,5559,4744,3624,4397,2296,1222,...,1612250,46237,53942,7,5196,"{""x"": -12989383.674504515, ""y"": 5413226.487333...",ID,16,1428,35856


> 注：詳細については、Pandas DataFrame [head() メソッドのドキュメント](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.head.html)を参照してください。

また、ArcGIS API for Python の [Feature Layer](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.features.toc.html#featurelayer) オブジェクトで、クエリを使用するとレコードのサブセットを返すこともできます。FeatureLayer に対して [query()](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.features.toc.html#arcgis.features.FeatureLayer.query) メソッドを実行すると、FeatureSet オブジェクトが返されます。FeatureSet の sdf プロパティを呼び出すと、Spatially Enabled DataFrame オブジェクトが返されます。次に、データフレームの [head()](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.GroupBy.head.html#pandas.core.groupby.GroupBy.head) メソッドを使用して、DataFrame から最初の 5 つのレコードとカラムのサブセットを返します。

#### 例：フィーチャ レイヤーのクエリ結果をSpatially Enabled DataFrame に送信する
AGE_45_54 カラムを使用してデータフレームに検索を行い、レコードのサブセットを含む新しい DataFrame を返します。[zip()](https://docs.python.org/3/library/functions.html#zip) 関数を使用してデータフレームの属性フィールド名を表示し、データフレーム構文を使用して出力の特定の属性フィールドを表示することができます。

In [6]:
# queryメソッドを使ってフィーチャレイヤーのレコードをフィルタリング

df = fl.query(where="AGE_45_54 < 1500").sdf

In [7]:
for a,b,c,d in zip(df.columns[::4], df.columns[1::4],df.columns[2::4], df.columns[3::4]):
    print("{:<30}{:<30}{:<30}{:<}".format(a,b,c,d))

FID                           NAME                          CLASS                         ST
STFIPS                        PLACEFIPS                     CAPITAL                       POP_CLASS
POPULATION                    POP2010                       WHITE                         BLACK
AMERI_ES                      ASIAN                         HAWN_PI                       HISPANIC
OTHER                         MULT_RACE                     MALES                         FEMALES
AGE_UNDER5                    AGE_5_9                       AGE_10_14                     AGE_15_19
AGE_20_24                     AGE_25_34                     AGE_35_44                     AGE_45_54
AGE_55_64                     AGE_65_74                     AGE_75_84                     AGE_85_UP
MED_AGE                       MED_AGE_M                     MED_AGE_F                     HOUSEHOLDS
AVE_HH_SZ                     HSEHLD_1_M                    HSEHLD_1_F                    MARHH_CHD
MARHH_NO_C   

In [8]:
# 最初の5つのレコードだけの列のサブセットを返す
df[['NAME', 'AGE_45_54', 'POP2010']].head()

,NAME,AGE_45_54,POP2010
0,Ammon,1446,13816
1,Blackfoot,1330,11899
2,Burley,1134,10345
3,Chubbuck,1494,13922
4,Jerome,1155,10890


## ローカルの GIS データへのアクセス
 
SEDF はローカルの GIS データにもアクセスできます。インストールされている Python のモジュールに応じて、幅広い機能にアクセスすることができます。

* ArcPy がインストールされている場合、つまり [ArcGIS Pro](http://pro.arcgis.com/en/pro-app/) がインストールされていて、同じ環境に ArcGIS API for Python がインストールされている場合、DataFrame には [ArcGIS Desktop がサポートする地理データ形式](http://desktop.arcgis.com/en/arcmap/10.3/manage-data/datatypes/about-geographic-data-formats.htm#ESRI_SECTION1_4835793C55C0439593A46FD5BC9E64B9)のサブセットを読み込む様々な方法があります。
 * [`フィーチャ クラス`](http://desktop.arcgis.com/en/arcmap/latest/manage-data/feature-classes/a-quick-tour-of-feature-classes.htm)
 * [`シェープファイル`](http://desktop.arcgis.com/en/arcmap/latest/manage-data/shapefiles/what-is-a-shapefile.htm)
 * [`ArcGIS Server Web Services`](https://enterprise.arcgis.com/en/server/latest/publish-services/windows/what-types-of-services-can-you-publish.htm), [`ArcGIS Online ホスト フィーチャ レイヤー`](https://doc.arcgis.com/en/arcgis-online/share-maps/publish-features.htm) 
 * [`OGC Services`](http://www.opengeospatial.org/standards)  

    
### 例：シェープファイルの読み込み
> ArcPy にアクセスできない Python インタプリタを使用してシェイプファイルを読み取る from_featureclass() メソッドを使用するには、ArcGIS Online または ArcGIS Enterprise で認証する必要があります。

>  `g2 = GIS("https://www.arcgis.com", "username", "password")`

In [9]:
g2 = GIS("https://arcgis.com", "<ユーザー名>", "<パスワード>")

In [8]:
sdf = pd.DataFrame.spatial.from_featureclass("path\to\your\data\census_example\cities.shp")
sdf.tail()

,FID,NAME,CLASS,ST,STFIPS,PLACEFIP,CAPITAL,AREALAND,AREAWATER,POP_CLASS,...,MARHH_NO_C,MHH_CHILD,FHH_CHILD,FAMILIES,AVE_FAM_SZ,HSE_UNITS,VACANT,OWNER_OCC,RENTER_OCC,SHAPE
3552,3552,East Providence,City,RI,44,22960,,13.405,3.208,6,...,5658,306,1414,12850,2.99,21309,779,12096,8434,"{'x': -71.3608270663031, 'y': 41.8015001782688..."
3553,3553,Pawtucket,City,RI,44,54640,,8.736,0.259,7,...,6740,754,3242,18520,3.07,31819,1772,13331,16716,"{'x': -71.3759815680945, 'y': 41.8755001649055..."
3554,3554,Fall River,City,MA,25,23000,,31.022,7.202,7,...,9011,759,4247,23558,3.00,41857,3098,13521,25238,"{'x': -71.1469910908576, 'y': 41.6981001567767..."
3555,3555,Somerset,Census Designated Place,MA,25,62465,,8.109,3.867,6,...,2771,91,287,5260,2.98,7143,156,5723,1264,"{'x': -71.15319106847441, 'y': 41.748500174901..."
3556,3556,New Bedford,City,MA,25,45000,,20.122,3.904,7,...,8813,910,4701,24083,3.01,41511,3333,16711,21467,"{'x': -70.93370908847608, 'y': 41.651800155406..."


### 例：ファイルジオデータベースからフィーチャ クラスを読み込む

> ArcPy にアクセスできない Python インタプリタで from_featureclass() メソッドを使って フファイルジオデータベースからフィーチャ クラスを読み込むには、fiona がインストールされている必要があります。

In [3]:
sdf = pd.DataFrame.spatial.from_featureclass("path\to\your\data\census_example\census.gdb\cities")
sdf.head()

,OBJECTID,FID,NAME,CLASS,ST,STFIPS,PLACEFIP,CAPITAL,AREALAND,AREAWATER,...,MARHH_NO_C,MHH_CHILD,FHH_CHILD,FAMILIES,AVE_FAM_SZ,HSE_UNITS,VACANT,OWNER_OCC,RENTER_OCC,SHAPE
0,1,0,College,Census Designated Place,AK,02,16750,,18.670,0.407,...,936,152,339,2640,3.13,4501,397,2395,1709,"{'x': -147.82719115699996, 'y': 64.84830019400..."
1,2,1,Fairbanks,City,AK,02,24230,,31.857,0.815,...,2259,395,1058,7187,3.15,12357,1282,3863,7212,"{'x': -147.72638162999996, 'y': 64.83809069700..."
2,3,2,Kalispell,City,MT,30,40075,,5.458,0.004,...,1433,147,480,3494,2.92,6532,390,3458,2684,"{'x': -114.31606412399998, 'y': 48.19780017900..."
3,4,3,Post Falls,City,ID,16,64810,,9.656,0.045,...,1851,205,467,4670,3.13,6697,328,4611,1758,"{'x': -116.93792709799999, 'y': 47.71555468000..."
4,5,4,Dishman,Census Designated Place,WA,53,17985,,3.378,0.000,...,1096,131,345,2564,2.96,4408,257,2635,1516,"{'x': -117.27780913799995, 'y': 47.65654568400..."


## Spatially Enabled DataFrame の保存

SEDFは、他のアプリケーションで使用するために、データを様々なデータ形式にエクスポートすることができます。

### エクスポート オプション

- [Feature Layers](https://doc.arcgis.com/en/arcgis-online/share-maps/hosted-web-layers.htm)
- [Feature Collections](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.features.toc.html#featurelayercollection)
- [Feature Set](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.features.toc.html#featureset)
- [GeoJSON](http://geojson.org/)
- [Feature Class](http://desktop.arcgis.com/en/arcmap/latest/manage-data/feature-classes/a-quick-tour-of-feature-classes.htm)
- [Pickle](https://pythontips.com/2013/08/02/what-is-pickle-in-python/)
- [HDF](https://support.hdfgroup.org/HDF5/Tutor/HDF5Intro.pdf)

### フィーチャ クラスへのエクスポート

SEDFでは、データセット全体または部分的なデータセットをエクスポートすることができます。

#### 例：データセット全体をシェープファイルにエクスポート

In [18]:
sdf.spatial.to_featureclass(location=r"c:\output_examples\census.shp")

'c:\\output_examples\\census.shp'

> ArcGIS API for Python をインストールした macOS および Linux マシン、ならびに ArcPy にアクセスできる Python インタプリタを使用していない Windows マシンでは、to_featureclass メソッドを使用してのみシェープファイル形式での書き出しが可能です。ファイルジオデータベースへの書き込みには ArcPy のサイトパッケージが必要です。

#### 例：列のサブセットと上位5つのレコードを含むデータセットをシェープファイルにエクスポート

In [17]:
for a,b,c,d in zip(sdf.columns[::4], sdf.columns[1::4], sdf.columns[2::4], sdf.columns[3::4]):
    print("{:<30}{:<30}{:<30}{:<}".format(a,b,c,d))

PLACENS                       GEOID                         NAMELSAD                      CLASSFP
FUNCSTAT                      ALAND                         AWATER                        INTPTLAT


In [15]:
columns = ['NAME', 'ST', 'CAPITAL', 'STFIPS', 'POP2000', 'POP2007', 'SHAPE']
sdf[columns].head().spatial.to_featureclass(location=r"/path/to/your/data/directory/sdf_head_output.shp")

'/path/to/your/data/directory/sdf_head_output.shp'

#### 例：データセットをファイルジオデータベースのフィーチャ クラスにエクスポート

In [23]:
sdf.spatial.to_featureclass(location=r"c:\output_examples\census.gdb\cities");

### フィーチャ レイヤーとして公開

SEDFでは、データセットをフィーチャ レイヤーとして公開することができます。  

#### 例：フィーチャ レイヤーとして公開

In [7]:
lyr = sdf.spatial.to_featurelayer('census_cities', folder='census')
lyr

<Item title:"census_cities" type:Feature Layer Collection owner:api_data_owner>